In [ ]:
!pip install ultralytics opencv-python numpy --quiet

import cv2
import os
import numpy as np
from ultralytics import YOLO
from collections import deque
import heapq

# ==========================
# CONFIG
# ==========================
VIDEO_SOURCE = r"C:\Users\deadbrat\Desktop\Shithi\T1.mp4"   # Your Dhaka city video
VIDEO_OUT = os.path.join(os.path.expanduser("~"), "Desktop", "dhaka352_traffic_output.mp4")

# Load YOLO model
model = YOLO("yolov8m.pt")

# ==========================
# SEARCH ALGORITHMS
# ==========================
def bfs(graph, start, goal):
    visited, queue = set(), deque([(start, [start])])
    while queue:
        node, path = queue.popleft()
        if node == goal:
            return path
        if node not in visited:
            visited.add(node)
            for neighbor in graph.get(node, []):
                queue.append((neighbor, path + [neighbor]))
    return None

def dfs(graph, start, goal):
    visited, stack = set(), [(start, [start])]
    while stack:
        node, path = stack.pop()
        if node == goal:
            return path
        if node not in visited:
            visited.add(node)
            for neighbor in graph.get(node, []):
                stack.append((neighbor, path + [neighbor]))
    return None

def ucs(graph, start, goal):
    pq = [(0, start, [start])]
    visited = set()
    while pq:
        cost, node, path = heapq.heappop(pq)
        if node == goal:
            return path, cost
        if node not in visited:
            visited.add(node)
            for neighbor, weight in graph.get(node, []):
                heapq.heappush(pq, (cost + weight, neighbor, path + [neighbor]))
    return None, float("inf")

# ==========================
# HELPER FUNCTIONS
# ==========================
def draw_futuristic_box(frame, x1, y1, x2, y2, label="", color=(0,255,255)):
    """Draws a neon sci-fi style bounding box with label"""
    overlay = frame.copy()
    cv2.rectangle(overlay, (x1, y1), (x2, y2), color, 2)
    cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
    cv2.rectangle(frame, (x1, y1-25), (x1+150, y1), (0,0,0), -1)
    cv2.putText(frame, label, (x1+5, y1-7),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    return frame

def build_graph(centers):
    """Create adjacency list graph from bounding box centers"""
    graph = {}
    for i, c1 in enumerate(centers):
        graph[i] = []
        for j, c2 in enumerate(centers):
            if i != j:
                dist = np.linalg.norm(np.array(c1) - np.array(c2))
                graph[i].append((j, dist))   # for UCS
    return graph

# ==========================
# MAIN
# ==========================
def main():
    cap = cv2.VideoCapture(VIDEO_SOURCE)
    if not cap.isOpened():
        print("❌ Error: Cannot open video file.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 25
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (width, height))

    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose=False)[0]

        centers, boxes = [], []
        for box in results.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = f"{model.names[cls_id]} {conf:.2f}"
            frame = draw_futuristic_box(frame, x1, y1, x2, y2, label)
            centers.append(((x1+x2)//2, (y1+y2)//2))
            boxes.append((x1, y1, x2, y2))

        # Apply BFS/DFS/UCS on graph of detected objects
        if len(centers) >= 2:
            graph = build_graph(centers)
            start, goal = 0, len(centers)-1  # first -> last vehicle

            bfs_path = bfs({k:[n for n,_ in v] for k,v in graph.items()}, start, goal)
            dfs_path = dfs({k:[n for n,_ in v] for k,v in graph.items()}, start, goal)
            ucs_path, ucs_cost = ucs(graph, start, goal)

            cv2.putText(frame, f"BFS Path: {bfs_path}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
            cv2.putText(frame, f"DFS Path: {dfs_path}", (10,60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)
            cv2.putText(frame, f"UCS Path: {ucs_path}, Cost={ucs_cost:.1f}", (10,90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)

        writer.write(frame)

        if frame_num % 50 == 0:
            print(f"Processed frame {frame_num}")
        frame_num += 1

    cap.release()
    writer.release()
    print(f"✅ Processing complete! Output saved to {VIDEO_OUT}")

main()


Processed frame 0
Processed frame 50
Processed frame 100
